# Online Retail Analytics Pipeline & Metrics

This notebook captures the key steps used to:

1. Install dependencies
2. Run the data pipeline to build `retail.duckdb`
3. Create the metrics layer views in DuckDB
4. Perform sanity checks on the warehouse
5. Launch the Streamlit dashboard

The interactive dashboard itself is implemented in `dashboard_app.py` and is normally run from a terminal with:

```bash
streamlit run dashboard_app.py
```

## 1. Environment & Project Layout

This notebook assumes that:

- You have this notebook file and the project files in the same folder, e.g.:

```text
analytics-assessment/
├── pipeline.py
├── metrics_definitions.sql
├── metrics_documentation.md
├── dashboard_app.py
├── requirements.txt
└── analytics_pipeline.ipynb   # this notebook
```

- You are running Python 3.10+ (tested with 3.12).
- The working directory in Jupyter is the project root (`analytics-assessment`).

## 2. Install Python Dependencies

This cell installs all required Python packages listed in `requirements.txt` using `pip`.

If you are using a virtual environment, make sure it is activated before launching Jupyter.

## 2. Install Python Dependencies (run outside the notebook)

To install the required Python packages, run the following in a terminal from the
project root (`analytics-assessment`):

```bash

cd path\to\analytics-assessment

cd %USERPROFILE%\Desktop\analytics-assessment

python -m venv venv

venv\Scripts\activate

pip install -r requirements.txt


## 3. Run the Data Pipeline

The pipeline script (`pipeline.py`) performs the following steps:

- Downloads the UCI Online Retail Excel file
- Cleans and filters the raw data
- Computes:
  - `line_revenue = Quantity * UnitPrice`
  - `invoice_month` (month-level date)
- Writes a DuckDB database `retail.duckdb` with two tables:
  - `raw_online_retail`
  - `fct_transactions`

In [1]:
import pathlib
import os
import importlib
import traceback

project_root = pathlib.Path.cwd()
pipeline_path = project_root / "pipeline.py"
db_path = project_root / "retail.duckdb"

print("Project root:", project_root)
print("Looking for pipeline.py at:", pipeline_path)

if not pipeline_path.exists():
    raise FileNotFoundError(f"pipeline.py not found at {pipeline_path}")

# If the DuckDB file already exists, don't rerun the pipeline.
if db_path.exists():
    print(f"\n✅ '{db_path.name}' already exists at {db_path}.")
    print("Skipping pipeline run.")
else:
    print("\n'retail.duckdb' not found. Attempting to run pipeline.main() in this environment...\n")
    try:
        import pipeline
        importlib.reload(pipeline)  # in case the file was edited
        pipeline.main()
        print("\n✅ Pipeline completed. DuckDB file 'retail.duckdb' should now exist.")
    except Exception as e:
        print("⚠️ Error while running pipeline.main() from this notebook.\n")
        print("Details:")
        traceback.print_exception(type(e), e, e.__traceback__)
        print("\nIf this fails due to environment issues, you can always run the pipeline from a terminal instead:")
        print("    python pipeline.py")


Project root: C:\Users\karol\Desktop\analytics-assessment
Looking for pipeline.py at: C:\Users\karol\Desktop\analytics-assessment\pipeline.py

✅ 'retail.duckdb' already exists at C:\Users\karol\Desktop\analytics-assessment\retail.duckdb.
Skipping pipeline run.


## 4. Create the Metrics Layer (SQL Views)

The metrics layer is defined in `metrics_definitions.sql` as a set of DuckDB views on top of `fct_transactions`, including:

- `order_revenue`
- `mrr_monthly`
- `customer_monthly_active`
- `customer_retention_monthly`
- `aov_overall`
- `aov_monthly`
- `customer_lifetime_value`
- `top_products_by_revenue`
- `revenue_by_country_monthly`

This cell executes the SQL file against `retail.duckdb` to create those views.

In [2]:
!pip install duckdb

import pathlib
import traceback

try:
    import duckdb
except ImportError:
    print("⚠️ duckdb is not installed in this Jupyter environment.")
    print("This step requires DuckDB. You can either:")
    print("  - Run this notebook from the virtualenv where DuckDB 1.x is installed, or")
    print("  - Install a recent DuckDB version (1.1.0+) in this environment.")
else:
    db_path = "retail.duckdb"
    sql_path = pathlib.Path("metrics_definitions.sql")

    print("Using DuckDB file:", db_path)
    print("Using SQL file:", sql_path)

    if not sql_path.exists():
        raise FileNotFoundError(f"metrics_definitions.sql not found at {sql_path}")

    try:
        con = duckdb.connect(db_path)
        with open(sql_path, "r") as f:
            sql_text = f.read()
        con.execute(sql_text)
        con.close()
        print("\n✅ Metrics views created successfully in DuckDB.")
    except Exception as e:
        print("⚠️ Error while connecting to or updating the DuckDB database.\n")
        traceback.print_exception(type(e), e, e.__traceback__)
        print("\nMost likely cause: this environment has an older DuckDB version")
        print("than the one that created 'retail.duckdb'.")
        print("Please run this notebook from the same environment where the pipeline was executed.")


Using DuckDB file: retail.duckdb
Using SQL file: metrics_definitions.sql

✅ Metrics views created successfully in DuckDB.


## 5. Sanity Checks on Tables and Views

This section runs a few quick checks to verify that:

- The DuckDB database contains the expected tables/views
- The `fct_transactions` fact table looks reasonable
- The `mrr_monthly` view returns data

In [3]:
import duckdb

con = duckdb.connect("retail.duckdb")

print("Tables/views in DuckDB:")
print(con.execute("SHOW TABLES;").df())

print("\nSample from fct_transactions:")
print(con.execute("SELECT * FROM fct_transactions LIMIT 5;").df())

print("\nSample from mrr_monthly:")
try:
    print(con.execute("SELECT * FROM mrr_monthly LIMIT 5;").df())
except Exception as e:
    print("Error querying mrr_monthly:", e)

con.close()

Tables/views in DuckDB:
                          name
0                  aov_monthly
1                  aov_overall
2      customer_lifetime_value
3      customer_monthly_active
4   customer_retention_monthly
5             fct_transactions
6                  mrr_monthly
7                order_revenue
8            raw_online_retail
9   revenue_by_country_monthly
10     top_products_by_revenue

Sample from fct_transactions:
  invoice_no stock_code                          description  quantity  \
0     536365     85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1     536365      71053                  WHITE METAL LANTERN         6   
2     536365     84406B       CREAM CUPID HEARTS COAT HANGER         8   
3     536365     84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4     536365     84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

    invoice_timestamp invoice_month  unit_price  line_revenue  customer_id  \
0 2010-12-01 08:26:00    2010-12-31        2.55   

## 6. Launching the Streamlit Dashboard

The interactive dashboard is defined in `dashboard_app.py`.  
It can be launched from a terminal or directly from this notebook.

### 6.1. Launch from this notebook (simple way)

Run the following in a **code cell** to start Streamlit:

```python
!streamlit run dashboard_app.py --server.runOnSave=false
```

- This will start a local web server and open the dashboard in your browser (e.g. at `http://localhost:8501`).
- The notebook cell will keep running while Streamlit is active.
- To stop Streamlit, interrupt the cell (■ Stop button or `Kernel → Interrupt`).

### 6.2. Launch in the background (optional)

If you prefer to keep using the notebook while Streamlit runs in the background, you can use:

```python
import subprocess
subprocess.Popen(["streamlit", "run", "dashboard_app.py"])
```

To stop the background process on Windows:

```python
!taskkill /IM streamlit.exe /F
```

### Alternative Streamlit dashboard steps

In [ ]:
# 1. Open a regular Command Prompt (NOT Anaconda Prompt)

# 2. Navigate to your project folder
cd C:\Users\karol\Desktop\analytics-assessment

# 3. Activate your virtual environment (the one where everything works)
venv\Scripts\activate

# You should now see:
# (venv) C:\Users\karol\Desktop\analytics-assessment>

# 4. (Optional) Install Streamlit in this venv if needed
pip install streamlit

# 5. Run the Streamlit dashboard
streamlit run dashboard_app.py

# 6. Open the dashboard in your browser:
# http://localhost:8501

# 7. To stop the dashboard:
# Press CTRL + C in the terminal
